In [1]:
import spacy

In [2]:
%%bash
python -m spacy download en_core_web_sm es_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.3.0/en_core_web_sm-3.3.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
doc = nlp("Time flies like an arrow")
for token in doc:
    print(token.text," / ",token.pos_)

Time  /  NOUN
flies  /  VERB
like  /  ADP
an  /  DET
arrow  /  NOUN
